In [33]:
import easyocr
import regex as re
import pandas as pd

In [34]:
reader = easyocr.Reader(['en'])

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [35]:
result1 = reader.readtext(r"C:\Users\Admin\Downloads\1.png",detail=0)
print(result1)

['Selva', 'DATA MANAGER', '+123-456-7890', '+123-456-7891', 'WWW XYZI.com', 'hello@XYZ1.com', '123 ABC St , Chennai;', 'selva', 'TamilNadu 600113', 'digitals']


In [36]:
result2 = reader.readtext(r"C:\Users\Admin\Downloads\2.png",detail=0)
print(result2)

['Amit kumar', 'CEO & FOUNDER', '123-456-7569', 'hello@global.com', 'WWW', 'global.com', '123 global', 'Erode,', 'GLOBAL', 'TamilNadu 600115', 'INSURANCE', 'St ,']


In [37]:
result3 = reader.readtext(r"C:\Users\Admin\Downloads\3.png",detail=0)
print(result3)

['KARTHICK', 'General Manager', '123 ABC St , Salem,', 'TamilNadu 6004513', '+123-456-7890', 'hello@Borcelle.com', 'www Borcelle.com', 'BORCELLE', 'AIRLINES']


In [38]:
result4 = reader.readtext(r"C:\Users\Admin\Downloads\4.png",detail=0)
print(result4)

['REVANTH', 'Marketing Executive', '123 ABC St,, HYDRABAD, TamilNadu;', '600001', '+91-456-1234', 'hello@CHRISTMAS.com', 'Family', 'wWW.CHRISTMAS.com', 'Restaurant']


In [39]:
result5 = reader.readtext(r"C:\Users\Admin\Downloads\5.png",detail=0)
print(result5)

['SANTHOSH', 'Technical Manager', '123 ABC St , Tirupur; TamilNadu,', '641603', '+123-456-1234', 'hello@Sun.com', 'www.Suncom', 'Sun Electricals']


In [40]:
def phno(img):
  num = []
  for i in img:
    if re.findall(r'^[+]',i):
      num.append(i)
    elif(re.findall(r'^\d{3}-\d{3}-\d{4}$',i)):
      num.append(i)
  return num

In [41]:
def email(img):
  for i in img:
    if(re.findall(r'[\w\.-]+@[\w\.-]+',i)):
      return i

In [42]:
def website(img):
  website = ""
  for i in img:
    if re.match(r'^WWW(?=.*\.com)', i):
      website = i
    elif re.match(r'^\w+\.com$', i):
      website = ('WWW.'+i)
  if len(website) ==0:
    website = "Not Available"
  else:
    return website


In [43]:
def name(img):
  for i in img:
    return i

In [44]:
def designation(img):
  for i in img:
    return img[1]


In [45]:
def address(img):
  for i in img:
    if(re.findall(r'^123+\s[\w\.-]+',i)):
      return i[0:10]


In [46]:
def domain(img):
  for i in img[-1]:
    if len(img[-1])> 5:
      return img[-1]
    else:
      return img[-2]


In [47]:
def district(img):
  for i in img:
    if(re.search(r'^123+\s',i)):
      if len(i[10:20])> 6:
        return i[11:20].replace(",","")
    elif (re.search(r'\bErode\b',i)):
      return i.replace(";","")
  return "Not Available"

In [48]:
def pincode(img):
    pincode = None
    for i in img:
        pincode_match = re.search(r'(\d{6})|\b(\d{3}\s*\d{3})\b', i)
        if pincode_match:
            pincode = pincode_match.group(0).replace(' ', '')
    return pincode

In [49]:
def state(img): 
    for i in img:
        match = re.search(r'TamilNadu', i)
        if match:
            return match.group()
    return "Not found"

In [50]:
#create a dictionary with all the data
def data(img):
  data = {}
  data['Name'] = name(img)
  data['Designation'] = designation(img)
  data['Domain'] = domain(img)
  data['Contact'] = phno(img)
  data['E-mail'] = email(img)
  data['Website'] = website(img)
  data['Address'] = address(img)
  data['District'] = district(img)
  data['State'] = state(img)
  data['Pincode'] = pincode(img)
  return data

In [51]:
df_1 = pd.DataFrame(data(result1))

In [52]:
df_2 = pd.DataFrame(data(result2))

In [53]:
df_3 = pd.DataFrame(data(result3))

In [54]:
df_4 = pd.DataFrame(data(result4))

In [55]:
df_5 = pd.DataFrame(data(result5))

In [56]:
df = pd.concat([df_1,df_2,df_3,df_4,df_5])

In [57]:
df.to_csv('Extracted_data.csv',index=False)

In [58]:
pd.read_csv('Extracted_data.csv')

,Name,Designation,Domain,Contact,E-mail,Website,Address,District,State,Pincode
0,Selva,DATA MANAGER,digitals,+123-456-7890,hello@XYZ1.com,WWW XYZI.com,123 ABC St,Chennai,TamilNadu,600113
1,Selva,DATA MANAGER,digitals,+123-456-7891,hello@XYZ1.com,WWW XYZI.com,123 ABC St,Chennai,TamilNadu,600113
2,Amit kumar,CEO & FOUNDER,INSURANCE,123-456-7569,hello@global.com,WWW.global.com,123 global,"Erode,",TamilNadu,600115
3,KARTHICK,General Manager,AIRLINES,+123-456-7890,hello@Borcelle.com,NaN,123 ABC St,Salem,TamilNadu,600451
4,REVANTH,Marketing Executive,Restaurant,+91-456-1234,hello@CHRISTMAS.com,NaN,123 ABC St,HYDRABA,TamilNadu,600001
5,SANTHOSH,Technical Manager,Sun Electricals,+123-456-1234,hello@Sun.com,NaN,123 ABC St,Tirupur,TamilNadu,641603
